In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Data Loading 

In [ ]:
sales_df = pd.read_csv('../datasets/predict-sales/sales_train.csv')
items_df = pd.read_csv('../datasets/predict-sales/items.csv')
test_df = pd.read_csv('../datasets/predict-sales/test.csv')

In [ ]:
sales_df.drop(labels=['date'],inplace=True,axis=1)
sales_df = sales_df.reset_index()
items_df.drop(labels=['item_name'],inplace=True,axis=1)

In [ ]:
dict_aux = {}

In [ ]:
def setPair(x, d):
    i = str(x[0]) + '-' + str(x[1])
    try:
        return d[i]
    except:
        result = i
        d[i] = result
        return result

In [ ]:
sales_df['ID_pair'] = sales_df[['shop_id','item_id']].apply(setPair, args=[dict_aux], axis=1)

In [ ]:
sales_df = sales_df.merge(items_df)

In [ ]:
dict_aux = {}

In [ ]:
sales_df['ID_CAT_pair'] = sales_df[['shop_id','item_category_id']].apply(setPair, args=[dict_aux], axis=1)

In [ ]:
sales_df.info()

In [ ]:
sales_df.head()

In [ ]:
sales_df.to_csv('../datasets/predict-sales/sales_train_enriched.csv')

## Direct Load

In [7]:
sales_df = pd.read_csv('../datasets/predict-sales/sales_train_enriched.csv')

In [8]:
sales_df.drop(labels=['Unnamed: 0'], inplace=True, axis=1)

In [9]:
sales_df.head()

,index,date_block_num,shop_id,item_id,item_price,item_cnt_day,ID_pair,item_category_id,ID_CAT_pair
0,0,0,59,22154,999.0,1.0,59-22154,37,59-37
1,3270,0,24,22154,999.0,1.0,24-22154,37,24-37
2,17081,0,27,22154,999.0,1.0,27-22154,37,27-37
3,25918,0,25,22154,999.0,1.0,25-22154,37,25-37
4,25919,0,25,22154,999.0,1.0,25-22154,37,25-37


In [10]:
key_th = 40
sales_df_ensemble = sales_df[sales_df['item_category_id']>=40]
sales_df_stacking = sales_df[sales_df['item_category_id']<40]
print('ensemble size:',sales_df_ensemble.shape)
print('stacking size:',sales_df_stacking.shape)

ensemble size: (1527153, 9)
stacking size: (1408696, 9)


# Visualizing Data 

Let's visualize some data from the test set:

In [6]:
def drawTimeSerie(df, sample, n):
    for i in range(n):
        shop_id = sample.iloc[i]['shop_id']
        item_id = sample.iloc[i]['item_id']
        id_df = df[(df['shop_id'] == shop_id) & (df['item_id'] == item_id)]
        id_df_grouped = id_df[['date_block_num','item_cnt_day']].groupby('date_block_num').sum().reset_index()
        plt.figure(figsize=[10,n*5])
        plt.subplot(n,1,i+1)
        plt.plot(id_df_grouped['date_block_num'], id_df_grouped['item_cnt_day'],'*-',)
        plt.title(str(shop_id) + '-' + str(item_id))

In [ ]:
n_samples = 10
sample = sales_df_ensemble.sample(n=n_samples, axis=0)

In [ ]:
drawTimeSerie(sales_df_ensemble, sample, n_samples)


# Preparing Data with FeatureTools (testing)

In [11]:
import featuretools as ft

Let's evaluate the FT process 

## Pre-proccesing

In [ ]:
prev_month_list = [6,7,8]
month = 9

In [ ]:
sales_months_df = sales_df[sales_df['date_block_num'].isin(prev_month_list)]
sales_result_df = sales_df[sales_df['date_block_num'] == month]

In [ ]:
sales_months_df.drop(labels=['date_block_num','shop_id','item_id','item_category_id'], inplace=True, axis=1)
sales_result_df.drop(labels=['date_block_num','shop_id','item_id','item_category_id'], inplace=True, axis=1)

In [ ]:
sales_months_df.head()

## EntitySet Processing (Recommended by FeatureTools)

We will calculate 3 tables that we must join later:

### Categories Aggregation 

In [12]:
def getCatAgg(sales_months_df):
    es = ft.EntitySet(id="prediction_sales")
    es = es.entity_from_dataframe(entity_id='sales',dataframe=sales_months_df, index='index')
    es = es.normalize_entity(base_entity_id='sales',
                         new_entity_id='idsCat',
                         index='ID_CAT_pair')
    feature_matrix_idsCat, feature_defs_idsCat = ft.dfs(entityset=es, target_entity='idsCat')
    idsCat = feature_matrix_idsCat.reset_index()
    idsCat_agg = idsCat[['ID_CAT_pair','SUM(sales.item_cnt_day)',
                     'MEAN(sales.item_cnt_day)','MEAN(sales.item_price)',
                     'STD(sales.item_cnt_day)','STD(sales.item_price)',
                     'MAX(sales.item_cnt_day)','MAX(sales.item_price)',
                     'MIN(sales.item_cnt_day)','MIN(sales.item_price)',
                     'SKEW(sales.item_cnt_day)','SKEW(sales.item_price)'
                    ]]
    idsCat_agg.columns = ['ID_CAT_pair','sum_shop_cat_sales',
                      'mean_shop_cat_day','mean_shop_cat_item_price',
                      'std_shop_cat_day','std_shop_cat_item_price',
                      'max_shop_cat_day','max_shop_cat_item_price',
                      'min_shop_cat_day','min_shop_cat_item_price',
                      'skew_shop_cat_day','skew_shop_cat_item_price',
                     ]
    return idsCat_agg

In [ ]:
idsCat = getCatAgg(sales_months_df)

In [ ]:
idsCat.head()

### Item Aggregation

In [13]:
def getItemAgg(sales_months_df):
    es = ft.EntitySet(id="prediction_sales")
    es = es.entity_from_dataframe(entity_id='sales',dataframe=sales_months_df, index='index')
    es = es.normalize_entity(base_entity_id='sales',
                         new_entity_id='ids',
                         index='ID_pair',
                         additional_variables=['ID_CAT_pair'])
    feature_matrix_ids, feature_defs_ids = ft.dfs(entityset=es, target_entity='ids')
    ids = feature_matrix_ids.reset_index()
    return ids

In [ ]:
ids = getItemAgg(sales_months_df)

In [ ]:
ids.head()

### Result aggregation 

In [14]:
def getTargetAgg(sales_result_df):
    es = ft.EntitySet(id="target_sales")
    es = es.entity_from_dataframe(entity_id='sales',dataframe=sales_result_df, index='index')
    es = es.normalize_entity(base_entity_id='sales',
                         new_entity_id='target',
                         index='ID_pair',
                         additional_variables=['ID_CAT_pair'])
    feature_matrix_target, feature_defs_target = ft.dfs(entityset=es, target_entity='target')
    target = feature_matrix_target.reset_index()
    target_agg = target[['ID_pair','ID_CAT_pair','SUM(sales.item_cnt_day)']]
    target_agg.columns = ['ID_pair','ID_CAT_pair','total_sales']
    return target_agg

In [ ]:
target = getTargetAgg(sales_result_df)

In [ ]:
target.head()

### Join of the 3 previous tables 

In [15]:
def joinThreeParts(ids, idsCat, target):
    df = ids.merge(right=idsCat,on='ID_CAT_pair',how='left').merge(right=target,on='ID_pair',how='outer')
    df.drop(labels=['ID_CAT_pair_x'], inplace=True, axis=1)
    df.columns = ['ID_pair', 'SUM(sales.item_price)', 'SUM(sales.item_cnt_day)',
       'STD(sales.item_price)', 'STD(sales.item_cnt_day)',
       'MAX(sales.item_price)', 'MAX(sales.item_cnt_day)',
       'SKEW(sales.item_price)', 'SKEW(sales.item_cnt_day)',
       'MIN(sales.item_price)', 'MIN(sales.item_cnt_day)',
       'MEAN(sales.item_price)', 'MEAN(sales.item_cnt_day)', 'COUNT(sales)',
       'sum_shop_cat_sales', 'mean_shop_cat_day', 'mean_shop_cat_item_price',
       'std_shop_cat_day', 'std_shop_cat_item_price', 'max_shop_cat_day',
       'max_shop_cat_item_price', 'min_shop_cat_day',
       'min_shop_cat_item_price', 'skew_shop_cat_day',
       'skew_shop_cat_item_price', 'ID_CAT_pair', 'total_sales']
    df_with_ids = df[~df['SUM(sales.item_price)'].isna()]
    df_without_ids = df[df['SUM(sales.item_price)'].isna()]
    df_without_ids.drop(labels=['sum_shop_cat_sales', 'mean_shop_cat_day', 'mean_shop_cat_item_price',
       'std_shop_cat_day', 'std_shop_cat_item_price', 'max_shop_cat_day',
       'max_shop_cat_item_price', 'min_shop_cat_day',
       'min_shop_cat_item_price', 'skew_shop_cat_day',
       'skew_shop_cat_item_price'], inplace=True, axis=1)
    df_without_ids_enriched = df_without_ids.merge(right=idsCat, on='ID_CAT_pair', how='inner')
    df_without_ids_enriched_sorted = df_without_ids_enriched[['ID_pair','SUM(sales.item_price)','SUM(sales.item_cnt_day)','STD(sales.item_price)','STD(sales.item_cnt_day)','MAX(sales.item_price)','MAX(sales.item_cnt_day)','SKEW(sales.item_price)','SKEW(sales.item_cnt_day)','MIN(sales.item_price)','MIN(sales.item_cnt_day)','MEAN(sales.item_price)','MEAN(sales.item_cnt_day)','COUNT(sales)','sum_shop_cat_sales','mean_shop_cat_day','mean_shop_cat_item_price','std_shop_cat_day','std_shop_cat_item_price','max_shop_cat_day','max_shop_cat_item_price','min_shop_cat_day','min_shop_cat_item_price','skew_shop_cat_day','skew_shop_cat_item_price','ID_CAT_pair','total_sales']]
    df_completed = pd.concat(objs=[df_with_ids,df_without_ids_enriched_sorted], axis=0)
    return df_completed

In [ ]:
df_joined = joinThreeParts(ids, idsCat, target)

In [ ]:
df_joined.shape

In [ ]:
sns.heatmap(~df_joined.isna())

# Generating the DataSet 

In [16]:
slots = sales_df['date_block_num'].nunique()

In [20]:
def generateFeatures(sales_df, months_feature, month_target, calculateTarget=True):
    print('features window:',months_feature,', target:',month_target)
    sales_months_df = sales_df[sales_df['date_block_num'].isin(months_feature)]
    sales_result_df = sales_df[sales_df['date_block_num'] == month_target]
    sales_months_df.drop(labels=['date_block_num','shop_id','item_id','item_category_id'], inplace=True, axis=1)
    sales_result_df.drop(labels=['date_block_num','shop_id','item_id','item_category_id'], inplace=True, axis=1)
    
    idsCat = getCatAgg(sales_months_df)
    ids = getItemAgg(sales_months_df)
    target = getTargetAgg(sales_result_df)
    
    joined = joinThreeParts(ids, idsCat, target)
    
    # Insert the slot component for correlation purposes
    joined['slot'] = joined.apply(lambda x: months_feature[-1])
    return joined

In [21]:
def slidingWindow(sales_df, size, slots):
    df_final = pd.DataFrame()
    for index in range(size,slots):
        features_target = generateFeatures(sales_df, np.arange(index-size,index),index)
        # TODO stack dataset
        df_final = pd.concat([df_final,features_target], axis=0) 
    # TODO return completed dataset
    return df_final

## Ensemble Dataset

### Dataset Test

In [22]:
dstest = slidingWindow(sales_df_ensemble, 3,6)

features window: [0 1 2] , target: 3
features window: [1 2 3] , target: 4
features window: [2 3 4] , target: 5


In [ ]:
windows = [3,6,12,18,25,32]

### Dataset 1

In [ ]:
dataset1A = slidingWindow(sales_df_ensemble, windows[0], slots)
dataset1B = slidingWindow(sales_df_stacking, windows[0], slots)

In [ ]:
dataset1A.to_csv('../datasets/predict-sales/dataset1A.csv')
dataset1B.to_csv('../datasets/predict-sales/dataset1B.csv')

### Dataset 2

In [ ]:
dataset2A = slidingWindow(sales_df_ensemble, windows[1], slots)
dataset2B = slidingWindow(sales_df_stacking, windows[1], slots)

In [ ]:
dataset2A.to_csv('datasets/predict-sales/dataset2A.csv')
dataset2B.to_csv('datasets/predict-sales/dataset2B.csv')

### Dataset 3

In [ ]:
dataset3A = slidingWindow(sales_df_ensemble, windows[2], slots)
dataset3B = slidingWindow(sales_df_stacking, windows[2], slots)

In [ ]:
dataset3A.to_csv('../datasets/predict-sales/dataset3A.csv')
dataset3B.to_csv('../datasets/predict-sales/dataset3B.csv')

### Dataset 4

In [ ]:
dataset4A = slidingWindow(sales_df_ensemble, windows[3], slots)
dataset4B = slidingWindow(sales_df_stacking, windows[3], slots)

In [ ]:
dataset4A.to_csv('../datasets/predict-sales/dataset4A.csv')
dataset4B.to_csv('../datasets/predict-sales/dataset4B.csv')

### Dataset 5

In [ ]:
dataset5A = slidingWindow(sales_df_ensemble, windows[4],slots)
dataset5B = slidingWindow(sales_df_stacking, windows[4],slots)

In [ ]:
dataset5A.to_csv('../datasets/predict-sales/dataset5A.csv')
dataset5B.to_csv('../datasets/predict-sales/dataset5B.csv')

### Dataset 6

In [ ]:
dataset6A = slidingWindow(sales_df_ensemble, windows[5],slots)
dataset6B = slidingWindow(sales_df_stacking, windows[5],slots)

In [ ]:
dataset6A.to_csv('../datasets/predict-sales/dataset6A.csv')
dataset6B.to_csv('../datasets/predict-sales/dataset6B.csv')

## Stacking Dataset 

In [ ]:
# TODO

# End of Case! 